# Assignment

**1. Interpretion and signficance**

Suppose that we would like to know how much families in the US are spending on recreation annually. We estimated the following model:

* expenditure=873+0.0012annual_income+0.00002annual_income^2−223.57have_kids
 
expenditure is the annual spending on recreation in US dollars, annual_income is the annual income in US dollars and have_kids is a dummy variable indicating the families with children. Interpret the estimated coefficients. What additional statistics should be given in order to make sure that your interpretations make sense statistically. Write up your answer and save in a Gist.

Solution:
* have_kids is a boolean(dummy) variable which have 0 or 1
* annual_income can change 0 to huge numbers

Additional:

* Maybe we can look p-values of these variables.

**2- Weather Model**

* First, load the dataset from the weatherinszeged table from Kaggle.
* Build a linear regression model where your target variable is the difference between the apparenttemperature and the temperature. As explanatory variables, use humidity and windspeed. Now, estimate your model using OLS. Are the estimated coefficients statistically significant? Are the signs of the estimated coefficients in line with your previous expectations? Interpret the estimated coefficients. What are the relations between the target and the explanatory variables?
* Next, include the interaction of humidity and windspeed to the model above and estimate the model using OLS. Are the coefficients statistically significant? Did the signs of the estimated coefficients for humidity and windspeed change? Interpret the estimated coefficients.

In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
import statsmodels.api as sm

import warnings
warnings.filterwarnings(action="ignore")

In [2]:
weather = pd.read_csv("data/weatherHistory.csv")
weather.head(5)

,Formatted Date,Summary,Precip Type,Temperature (C),Apparent Temperature (C),Humidity,Wind Speed (km/h),Wind Bearing (degrees),Visibility (km),Loud Cover,Pressure (millibars),Daily Summary
0,2006-04-01 00:00:00.000 +0200,Partly Cloudy,rain,9.472222,7.388889,0.89,14.1197,251.0,15.8263,0.0,1015.13,Partly cloudy throughout the day.
1,2006-04-01 01:00:00.000 +0200,Partly Cloudy,rain,9.355556,7.227778,0.86,14.2646,259.0,15.8263,0.0,1015.63,Partly cloudy throughout the day.
2,2006-04-01 02:00:00.000 +0200,Mostly Cloudy,rain,9.377778,9.377778,0.89,3.9284,204.0,14.9569,0.0,1015.94,Partly cloudy throughout the day.
3,2006-04-01 03:00:00.000 +0200,Partly Cloudy,rain,8.288889,5.944444,0.83,14.1036,269.0,15.8263,0.0,1016.41,Partly cloudy throughout the day.
4,2006-04-01 04:00:00.000 +0200,Mostly Cloudy,rain,8.755556,6.977778,0.83,11.0446,259.0,15.8263,0.0,1016.51,Partly cloudy throughout the day.


In [3]:
lst = []
for i in range(weather.shape[0]):
    lst.append(weather["Temperature (C)"].iloc[i]-weather["Apparent Temperature (C)"].iloc[i])

weather["target"] = lst

In [4]:
X = weather[["Humidity","Wind Speed (km/h)"]]
Y = weather["target"]

In [5]:
model = linear_model.LinearRegression()
model.fit(X,Y)

print("Coefficents : {}".format(model.coef_))
print("Intercept : {}".format(model.intercept_))

Coefficents : [3.02918594 0.11929075]
Intercept : -2.4381054151877017


In [6]:
X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.288
Model:                            OLS   Adj. R-squared:                  0.288
Method:                 Least Squares   F-statistic:                 1.949e+04
Date:                Thu, 17 Sep 2020   Prob (F-statistic):               0.00
Time:                        14:49:07   Log-Likelihood:            -1.7046e+05
No. Observations:               96453   AIC:                         3.409e+05
Df Residuals:                   96450   BIC:                         3.409e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
const                -2.4381      0.021   -115.948      0.000      -2.479      -2.397
Humidity              3.0292      0.024    126.479      0.000       2.982       3.076
Wind Speed (km/h)     0.1193      0.001    176.164      0.000       0.118       0.121
==============================================================================
Omnibus:                     3935.747   Durbin-Watson:                   0.264
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             4613.311
Skew:                           0.478   Prob(JB):                         0.00
Kurtosis:                       3.484   Cond. No.                         88.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

1- All variables are significant because their p-values are lower than 0.05. 

In [7]:
weather.Humidity.describe()

count    96453.000000
mean         0.734899
std          0.195473
min          0.000000
25%          0.600000
50%          0.780000
75%          0.890000
max          1.000000
Name: Humidity, dtype: float64

In [8]:
weather["Wind Speed (km/h)"].describe()

count    96453.000000
mean        10.810640
std          6.913571
min          0.000000
25%          5.828200
50%          9.965900
75%         14.135800
max         63.852600
Name: Wind Speed (km/h), dtype: float64

*Humidity have a coefficient 3.02918594 but Humidity has range between 0-1.*
*Wind Speed have a coefficient 0.11929075 but WindSpeed has range between 0-63.8 so when we just looked coefficents we think that humidity have bigger impact than wind speed. However windspeed have bigger max value and It directly effects the target value. When we product coefficents and features, we see that wind speed have more impact than Humidity.* 

In [9]:
X = weather[["Humidity","Wind Speed (km/h)"]]
lst = []
for i in range(len(weather)):
    lst.append(weather.Humidity.iloc[i]*weather["Wind Speed (km/h)"].iloc[i])

X["Humidity_Wind_speed"] = lst 
Y = weather["target"]

In [10]:
model = linear_model.LinearRegression()
model.fit(X,Y)

print("Coefficents : {}".format(model.coef_))
print("Intercept : {}".format(model.intercept_))

Coefficents : [-0.17751219 -0.09048213  0.29711946]
Intercept : -0.0839363100978221


In [11]:
X = sm.add_constant(X)
results = sm.OLS(Y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 target   R-squared:                       0.341
Model:                            OLS   Adj. R-squared:                  0.341
Method:                 Least Squares   F-statistic:                 1.666e+04
Date:                Thu, 17 Sep 2020   Prob (F-statistic):               0.00
Time:                        14:49:10   Log-Likelihood:            -1.6669e+05
No. Observations:               96453   AIC:                         3.334e+05
Df Residuals:                   96449   BIC:                         3.334e+05
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
const                  -0.0839      0.033     -2.511      0.012      -0.149      -0.018
Humidity               -0.1775      0.043     -4.133      0.000      -0.262      -0.093
Wind Speed (km/h)      -0.0905      0.002    -36.797      0.000      -0.095      -0.086
Humidity_Wind_speed     0.2971      0.003     88.470      0.000       0.291       0.304
==============================================================================
Omnibus:                     4849.937   Durbin-Watson:                   0.262
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             9295.404
Skew:                           0.378   Prob(JB):                         0.00
Kurtosis:                       4.320   Cond. No.                         193.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [12]:
# Coefficents -> Significant
# Normal coefficents are changed in this case.

X["Humidity"].describe()

count    96453.000000
mean         0.734899
std          0.195473
min          0.000000
25%          0.600000
50%          0.780000
75%          0.890000
max          1.000000
Name: Humidity, dtype: float64

In [13]:
X["Wind Speed (km/h)"].describe()

count    96453.000000
mean        10.810640
std          6.913571
min          0.000000
25%          5.828200
50%          9.965900
75%         14.135800
max         63.852600
Name: Wind Speed (km/h), dtype: float64

In [14]:
X["Humidity_Wind_speed"].describe()

count    96453.000000
mean         7.640729
std          5.034842
min          0.000000
25%          3.820852
50%          6.701464
75%         10.213840
max         43.346835
Name: Humidity_Wind_speed, dtype: float64

When we product the coefficents and feature max values. We can see that Humidity_Wind_speed have the biggest impact on the target value.

**3. House prices model**

* Load the houseprices data from Kaggle.
* Run your house prices model again and interpret the results. Which features are statistically significant and which are not?
* Now, exclude the insignificant features from your model. Did anything change?
* Interpret the statistically significant coefficients by quantifying their relations with the house prices. Which features have more prominent effect on the house prices?
* Do the results sound reasonable to you? If not, try to explain the potential reasons.

In [37]:
house_prices = pd.read_csv("data/houseprices.csv")
house = house_prices.select_dtypes(exclude='object')
house.isnull().sum()*100/house.shape[0]

MSSubClass        0.000000
LotFrontage      17.739726
LotArea           0.000000
OverallQual       0.000000
OverallCond       0.000000
YearBuilt         0.000000
YearRemodAdd      0.000000
MasVnrArea        0.547945
BsmtFinSF1        0.000000
BsmtFinSF2        0.000000
BsmtUnfSF         0.000000
TotalBsmtSF       0.000000
1stFlrSF          0.000000
2ndFlrSF          0.000000
LowQualFinSF      0.000000
GrLivArea         0.000000
BsmtFullBath      0.000000
BsmtHalfBath      0.000000
FullBath          0.000000
HalfBath          0.000000
BedroomAbvGr      0.000000
KitchenAbvGr      0.000000
TotRmsAbvGrd      0.000000
Fireplaces        0.000000
GarageYrBlt       5.547945
GarageCars        0.000000
GarageArea        0.000000
WoodDeckSF        0.000000
OpenPorchSF       0.000000
EnclosedPorch     0.000000
3SsnPorch         0.000000
ScreenPorch       0.000000
PoolArea          0.000000
MiscVal           0.000000
MoSold            0.000000
YrSold            0.000000
SalePrice         0.000000
d

In [44]:
X = house[[x for x in house.columns if x not in ["LotFrontage","SalePrice","GarageYrBlt","MasVnrArea"]]]

Y = house["SalePrice"]

In [45]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [47]:
model = linear_model.LinearRegression()
model.fit(X_train,y_train)

print("Coefficents : {}".format(model.coef_))
print("Intercept : {}".format(model.intercept_))

Coefficents : [-1.75548690e+02  4.09174234e-01  1.84955503e+04  3.83573710e+03
  3.37312908e+02  1.80209917e+02  1.00461555e+01 -4.61607099e+00
  3.82786747e-01  5.81287122e+00  1.31240039e+01  1.42134340e+01
  4.50422890e+00  3.18416668e+01  1.10492803e+04 -3.26518493e+02
  3.42606130e+03 -1.66531509e+03 -9.08810085e+03 -1.00971990e+04
  5.10100134e+03  4.37021557e+03  1.16074917e+04  5.58607730e-01
  2.55639049e+01 -6.62562003e+00  7.06908781e+00  3.85262374e+01
  6.55989735e+01 -3.73443286e+01 -7.15692563e-01 -2.09433750e+02
 -5.14524004e+02]
Intercept : -48411.54387291285


In [48]:
X_test = sm.add_constant(X_test)
results = sm.OLS(y_test,X_test).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.866
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                     54.04
Date:                Thu, 17 Sep 2020   Prob (F-statistic):           1.83e-95
Time:                        15:10:47   Log-Likelihood:                -3444.3
No. Observations:                 292   AIC:                             6953.
Df Residuals:                     260   BIC:                             7070.
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
const           4.32e+05   3.16e+06      0.137      0.891    -5.8e+06    6.66e+06
MSSubClass      -90.3671     60.490     -1.494      0.136    -209.479      28.745
LotArea           0.8979      0.403      2.230      0.027       0.105       1.691
OverallQual    1.392e+04   2801.257      4.970      0.000    8406.883    1.94e+04
OverallCond    7562.2941   2367.198      3.195      0.002    2900.973    1.22e+04
YearBuilt       271.1093    147.505      1.838      0.067     -19.348     561.566
YearRemodAdd      0.1949    157.715      0.001      0.999    -310.366     310.756
BsmtFinSF1       22.8507      5.732      3.987      0.000      11.564      34.137
BsmtFinSF2        5.7991      9.443      0.614      0.540     -12.796      24.394
BsmtUnfSF         0.3119      5.198      0.060      0.952      -9.924      10.548
TotalBsmtSF      28.9617      7.226      4.008      0.000      14.732      43.191
1stFlrSF         50.7316     13.410      3.783      0.000      24.325      77.138
2ndFlrSF         44.1333     12.616      3.498      0.001      19.290      68.976
LowQualFinSF    -63.4709     32.580     -1.948      0.052    -127.626       0.684
GrLivArea        31.3940     12.640      2.484      0.014       6.505      56.283
BsmtFullBath  -9932.6861   6126.607     -1.621      0.106    -2.2e+04    2131.399
BsmtHalfBath   5540.7283   8857.934      0.626      0.532   -1.19e+04     2.3e+04
FullBath       4900.2409   6315.559      0.776      0.439   -7535.915    1.73e+04
HalfBath        399.9268   6191.803      0.065      0.949   -1.18e+04    1.26e+04
BedroomAbvGr  -1.727e+04   3556.616     -4.856      0.000   -2.43e+04   -1.03e+04
KitchenAbvGr  -2.072e+04   1.28e+04     -1.624      0.106   -4.58e+04    4405.795
TotRmsAbvGrd   3021.1555   2670.327      1.131      0.259   -2237.065    8279.376
Fireplaces    -2423.8765   4062.648     -0.597      0.551   -1.04e+04    5576.006
GarageCars      1.11e+04   6271.765      1.770      0.078   -1249.130    2.35e+04
GarageArea       -5.4155     20.731     -0.261      0.794     -46.237      35.406
WoodDeckSF       55.2175     21.317      2.590      0.010      13.242      97.194
OpenPorchSF      69.5084     46.325      1.500      0.135     -21.712     160.729
EnclosedPorch   -23.5603     40.525     -0.581      0.561    -103.359      56.239
3SsnPorch      -212.2752    114.679     -1.851      0.065    -438.094      13.543
ScreenPorch      25.8486     38.777      0.667      0.506     -50.509     102.206
PoolArea         49.2722     60.560      0.814      0.417     -69.979     168.523
MiscVal          11.6799     27.792      0.420      0.675     -43.046      66.406
MoSold          912.7795    754.763      1.209      0.228    -573.447    2399.006
YrSold         -526.3356   1570.760     -0.335      0.738   -3619.367    2566.696
=================================================

**Significant Variables :** <br>
MSSubClass, LotArea , OverallQual, OverallCond, YearBuilt, BsmtFinSF1, TotalBsmtSF ,<br>
1stFlrSF, 2ndFlrSF, LowQualFinSF, GrLivArea , BedroomAbvGr, GarageCars, WoodDeckSF

In [50]:
X = house[["MSSubClass", "LotArea" , "OverallQual", "OverallCond", "YearBuilt", "BsmtFinSF1", "TotalBsmtSF","1stFlrSF", "2ndFlrSF", "LowQualFinSF", "GrLivArea" , "BedroomAbvGr","GarageCars","WoodDeckSF"]]

Y = house["SalePrice"]

In [51]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20, random_state=42)

In [52]:
model = linear_model.LinearRegression()
model.fit(X_train,y_train)

print("Coefficents : {}".format(model.coef_))
print("Intercept : {}".format(model.intercept_))

Coefficents : [-1.81745818e+02  4.59216953e-01  2.03977445e+04  4.88623602e+03
  4.06772811e+02  1.64385879e+01  3.16228998e+00  1.65382383e+01
  1.41905365e+01  1.16379198e+01  4.23666946e+01 -6.39007249e+03
  1.30755258e+04  2.66871646e+01]
Intercept : -874218.2486846152


In [53]:
X_test = sm.add_constant(X_test)
results = sm.OLS(y_test,X_test).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              SalePrice   R-squared:                       0.857
Model:                            OLS   Adj. R-squared:                  0.850
Method:                 Least Squares   F-statistic:                     127.9
Date:                Thu, 17 Sep 2020   Prob (F-statistic):          5.34e-109
Time:                        15:53:04   Log-Likelihood:                -3453.7
No. Observations:                 292   AIC:                             6935.
Df Residuals:                     278   BIC:                             6987.
Df Model:                          13                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const        -6.892e+05   2.11e+05     -3.260      0.001   -1.11e+06   -2.73e+05
MSSubClass    -127.0121     57.013     -2.228      0.027    -239.245     -14.779
LotArea          0.7917      0.386      2.050      0.041       0.031       1.552
OverallQual   1.607e+04   2540.140      6.325      0.000    1.11e+04    2.11e+04
OverallCond   7586.2815   2019.412      3.757      0.000    3611.000    1.16e+04
YearBuilt      296.7584    108.883      2.725      0.007      82.419     511.098
BsmtFinSF1      15.1313      5.752      2.631      0.009       3.808      26.454
TotalBsmtSF     33.0413      8.365      3.950      0.000      16.575      49.507
1stFlrSF        42.9968     12.206      3.523      0.000      18.969      67.025
2ndFlrSF        43.6313     11.331      3.851      0.000      21.326      65.937
LowQualFinSF   -47.0596     30.647     -1.536      0.126    -107.389      13.270
GrLivArea       39.5685     10.784      3.669      0.000      18.340      60.797
BedroomAbvGr -1.449e+04   3114.031     -4.653      0.000   -2.06e+04   -8358.820
GarageCars    1.042e+04   3761.105      2.770      0.006    3013.537    1.78e+04
WoodDeckSF      48.4558     20.262      2.391      0.017       8.569      88.343
==============================================================================
Omnibus:                      136.573   Durbin-Watson:                   2.037
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1356.140
Skew:                           1.634   Prob(JB):                    3.30e-295
Kurtosis:                      13.039   Cond. No.                     1.17e+17
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The smallest eigenvalue is 2.94e-24. This might indicate that there are
strong multicollinearity problems or that the design matrix is singular.
"""

* *There are changes in P-Values, some variables are decreased, others increased. There is no certain change in p-values.*

**Interpretion**

In [54]:
X.columns

Index(['MSSubClass', 'LotArea', 'OverallQual', 'OverallCond', 'YearBuilt',
       'BsmtFinSF1', 'TotalBsmtSF', '1stFlrSF', '2ndFlrSF', 'LowQualFinSF',
       'GrLivArea', 'BedroomAbvGr', 'GarageCars', 'WoodDeckSF'],
      dtype='object')

In [61]:
print("y = {:.2f} + {:.2f} {} + {:.2f}{} + {:.2f} {} + {:.2f} {} \n+ {:.2f} {} + {:.2f} {} + {:.2f} {} + {:.2f} {}\n + {:.2f} {} + {:.2f} {} + {:.2f} {}\n + {:.2f} {} + {:.2f} {} + {:.2f} {}".format(model.intercept_, model.coef_[0],X.columns[0], model.coef_[1],X.columns[1], model.coef_[2],X.columns[2], model.coef_[3],X.columns[3], model.coef_[4],X.columns[4], model.coef_[5],X.columns[5], model.coef_[6],X.columns[6], model.coef_[7],X.columns[7], model.coef_[8],X.columns[8], model.coef_[9],X.columns[9], model.coef_[10],X.columns[10], model.coef_[11],X.columns[11], model.coef_[12],X.columns[12], model.coef_[13],X.columns[13]))

y = -874218.25 + -181.75 MSSubClass + 0.46LotArea + 20397.74 OverallQual + 4886.24 OverallCond 
+ 406.77 YearBuilt + 16.44 BsmtFinSF1 + 3.16 TotalBsmtSF + 16.54 1stFlrSF
 + 14.19 2ndFlrSF + 11.64 LowQualFinSF + 42.37 GrLivArea
 + -6390.07 BedroomAbvGr + 13075.53 GarageCars + 26.69 WoodDeckSF


In [65]:
X.max()

MSSubClass         190
LotArea         215245
OverallQual         10
OverallCond          9
YearBuilt         2010
BsmtFinSF1        5644
TotalBsmtSF       6110
1stFlrSF          4692
2ndFlrSF          2065
LowQualFinSF       572
GrLivArea         5642
BedroomAbvGr         8
GarageCars           4
WoodDeckSF         857
dtype: int64

In [63]:
X.min()

MSSubClass        20
LotArea         1300
OverallQual        1
OverallCond        1
YearBuilt       1872
BsmtFinSF1         0
TotalBsmtSF        0
1stFlrSF         334
2ndFlrSF           0
LowQualFinSF       0
GrLivArea        334
BedroomAbvGr       0
GarageCars         0
WoodDeckSF         0
dtype: int64

In [71]:
for x in range(X.shape[1]):
    print("{} have max value : {:.2f} \n{} have min value: {:.2f}\n".format(X.columns[x],(model.coef_[x]*X.max().iloc[x]),X.columns[x],(model.coef_[x]*X.min().iloc[x])))

MSSubClass have max value : -34531.71 
MSSubClass have min value: -3634.92

LotArea have max value : 98844.15 
LotArea have min value: 596.98

OverallQual have max value : 203977.44 
OverallQual have min value: 20397.74

OverallCond have max value : 43976.12 
OverallCond have min value: 4886.24

YearBuilt have max value : 817613.35 
YearBuilt have min value: 761478.70

BsmtFinSF1 have max value : 92779.39 
BsmtFinSF1 have min value: 0.00

TotalBsmtSF have max value : 19321.59 
TotalBsmtSF have min value: 0.00

1stFlrSF have max value : 77597.41 
1stFlrSF have min value: 5523.77

2ndFlrSF have max value : 29303.46 
2ndFlrSF have min value: 0.00

LowQualFinSF have max value : 6656.89 
LowQualFinSF have min value: 0.00

GrLivArea have max value : 239032.89 
GrLivArea have min value: 14150.48

BedroomAbvGr have max value : -51120.58 
BedroomAbvGr have min value: -0.00

GarageCars have max value : 52302.10 
GarageCars have min value: 0.00

WoodDeckSF have max value : 22870.90 
WoodDeckSF ha

* *LotArea ,OverallCond , OverallQual , BsmtFinSF1 , GrLivArea have huge impacts.*

* *We have biggest impact from Ovearall Quallity Feature(OverallQual) and It can be expected. On the other hand, Area have important thing to when you want to buy a house.*